# Upload Parquet to Neon PostgreSQL (pandas)

Notebook preparado especificamente para subir o Parquet do case **Dadosfera** para o **Neon PostgreSQL**.


In [ ]:
# Instale dependências se necessário (rode apenas uma vez)
# !pip install pandas pyarrow sqlalchemy psycopg2-binary


## 1) Configuração de conexão e caminho do arquivo


In [1]:
from sqlalchemy import create_engine, text
import pandas as pd

# ⚠️ Connection string Neon (credenciais reais)
DB_URL = (
    "postgresql://neondb_owner:npg_ezao8l7ZCOBp@"
    "ep-summer-term-ad98j6jn-pooler.c-2.us-east-1.aws.neon.tech/"
    "neondb?sslmode=require&channel_binding=require"
)

# Caminho do Parquet
PARQUET_PATH = r"C:\Users\Rodrigo\Desktop\py\Prjt\DDF_TECH_122025\notebooks\data\electronics_reviews_prepared.parquet"

# Nome da tabela destino
TABLE_NAME = "electronics_reviews"

# Cria engine
engine = create_engine(DB_URL, pool_pre_ping=True)

# Teste rápido de conexão
with engine.connect() as conn:
    print("Teste conexão:", conn.execute(text("select 1")).scalar())


Teste conexão: 1


## 2) Leitura do Parquet


In [2]:
df = pd.read_parquet(PARQUET_PATH)
df.head(), df.shape


(       reviewerID        asin  \
 0   AO94DHGC771SJ  0528881469   
 1   AMO214LNFCEI4  0528881469   
 2  A3N7T0DY83Y4IG  0528881469   
 3  A1H8PY3QHMQQA0  0528881469   
 4  A24EV6RXELQZ63  0528881469   
 
                                           reviewText  overall  \
 0  We got this GPS for my husband who is an (OTR)...      5.0   
 1  I'm a professional OTR truck driver, and I bou...      1.0   
 2  Well, what can I say.  I've had this unit in m...      3.0   
 3  Not going to write a long review, even thought...      2.0   
 4  I've had mine for a year and here's what we go...      1.0   
 
                                   summary  unixReviewTime   reviewTime  \
 0                         Gotta have GPS!      1370131200   06 2, 2013   
 1                       Very Disappointed      1290643200  11 25, 2010   
 2                          1st impression      1283990400   09 9, 2010   
 3                 Great grafics, POOR GPS      1290556800  11 24, 2010   
 4  Major issues, onl

## 3) Normalização de colunas (Postgres-friendly)


In [3]:
# Normaliza nomes das colunas
df.columns = (
    df.columns.astype(str)
      .str.strip()
      .str.lower()
      .str.replace(r"\s+", "_", regex=True)
      .str.replace(r"[^a-z0-9_]", "", regex=True)
)

# Remove coluna 'helpful' (lista), pois já existe helpful_up / helpful_total
if "helpful" in df.columns:
    df = df.drop(columns=["helpful"])

# Converte data
if "review_datetime" in df.columns:
    df["review_datetime"] = pd.to_datetime(df["review_datetime"], errors="coerce")

df.head()


,reviewerid,asin,reviewtext,overall,summary,unixreviewtime,reviewtime,review_datetime,year,month,helpful_up,helpful_total,reviewtext_len
0,AO94DHGC771SJ,0528881469,We got this GPS for my husband who is an (OTR)...,5.0,Gotta have GPS!,1370131200,"06 2, 2013",2013-06-02,2013,6,0,0,805
1,AMO214LNFCEI4,0528881469,"I'm a professional OTR truck driver, and I bou...",1.0,Very Disappointed,1290643200,"11 25, 2010",2010-11-25,2010,11,12,15,2175
2,A3N7T0DY83Y4IG,0528881469,"Well, what can I say. I've had this unit in m...",3.0,1st impression,1283990400,"09 9, 2010",2010-09-09,2010,9,43,45,4607
3,A1H8PY3QHMQQA0,0528881469,"Not going to write a long review, even thought...",2.0,"Great grafics, POOR GPS",1290556800,"11 24, 2010",2010-11-24,2010,11,9,10,2246
4,A24EV6RXELQZ63,0528881469,I've had mine for a year and here's what we go...,1.0,"Major issues, only excuses for support",1317254400,"09 29, 2011",2011-09-29,2011,9,0,0,1076


## 4) Upload para o Neon PostgreSQL


In [4]:
CHUNKSIZE = 20_000

df.to_sql(
    TABLE_NAME,
    engine,
    if_exists="replace",  # recria tabela com schema do Parquet
    index=False,
    chunksize=CHUNKSIZE,
    method="multi",
)

print("✅ Upload concluído com sucesso")


✅ Upload concluído com sucesso


## 5) Validação final


In [5]:
with engine.connect() as conn:
    count = conn.execute(text(f"SELECT COUNT(*) FROM {TABLE_NAME}")).scalar()

print(f"Total de registros na tabela '{TABLE_NAME}': {count}")


Total de registros na tabela 'electronics_reviews': 400000
